Start with importing the SQLite3 library and connect to the database.

You do not necessarily have to have to add the .db extension but it is good practice to denote that it is a database file.

After creating the connection you need to create a "mouse" that will carry out your commands.

In [ ]:
import sqlite3

conn = sqlite3.connect('Bad_Data.db')

c = conn.cursor()

Now we can take a look at the database.

https://docs.google.com/spreadsheets/d/1YUe668NntxiGrT34o2klO8jyymeq3k3TWoUKJtFcUQM/edit?usp=sharing

The "Bad_Data" database only contains the "Original_Data" table and has a lot of formatting issues that we will explore.

### Scenario 1

A store manager wants to see what Jon bought on Day 1 as well as how much Jon spent in total on Day 1.

In [ ]:
REQUEST = ["Jon", "1"]

data = c.execute("SELECT Item_1, Quantity_1, Unit_Price_1, Item_2, Quantity_2, Unit_Price_2, Item_3, Quantity_3, Unit_Price_3 FROM Original_Data WHERE Customer_Name = ? AND Day = ?;", REQUEST)

data = data.fetchone()

ITEM = []
QUANTITY = []
UNIT_PRICE = []

for i in range(0, len(data), 3):
    ITEM.append(data[i])
    QUANTITY.append(data[i+1])
    UNIT_PRICE.append(data[i+2])
    
SUM = 0

for i in range(len(ITEM)):
    
    if ITEM[i] != None:
    
        SUM = SUM + QUANTITY[i] * UNIT_PRICE[i]

        print("Item name: " + str(ITEM[i]))
        print("Quantity: " + str(QUANTITY[i]))
        print("Unit Price: " + str(UNIT_PRICE[i]))

print()
print("Total Spent: " + str(SUM))

### Scenario 2

On day 3, Peter comes to the store and buys 1 Banana.

Let's record this in the database

In [ ]:
DATA = ["3", "Peter", "Banana", 1, "0.2"]

c.execute("INSERT INTO Original_Data(Day, Customer_Name, Item_1, Quantity_1, Unit_Price_1) VALUES (?,?,?,?,?)", DATA)

conn.commit()

### Scenario 3

Steve realised that he forgot to pay for a cucumber he bought on Day 1. He returns to the store to pay more it and now we need to update the database.

Except that we can't because our database is not designed to handle more than 3 items bought per day.

# Normalisation

Used to reduce data redundancy and increase data integrity

There are many steps to normalising a database which each step fulfilling specific conditions/goals.

Normalisation is a trade off so the level of normalisation required varies from case to case.

Now we will change our connection to "Store_Database.db"

The database contain the tables "Order_Record", "Customer_Detail" and "Product_Info"

In [ ]:
import sqlite3

conn = sqlite3.connect('Store_Database.db')

c = conn.cursor()

Let's attempt the same 3 scenarios as before.

This time, we will be writing functions instead of one off commands.

### Scenario 4

A store manager wants to see what (a customer) bought on (a day).

He also wants to see how much the customer spent in total on the day.

In [ ]:
def view_purchase(Customer_Name, Day):
    Customer_ID = c.execute("SELECT Customer_ID FROM Customer_Detail WHERE Name = ?", (Customer_Name,))
    Customer_ID = Customer_ID.fetchone()
    Customer_ID = Customer_ID[0]
    
    data = c.execute("SELECT Product_ID, Quantity FROM Order_Record WHERE Customer_ID = ? AND Day = ?;", (Customer_ID, Day))
    data = data.fetchall()
    
    PRODUCT_IDs = []
    QUANTITIES = []
    
    for i in range(len(data)):
        PRODUCT_IDs.append(data[i][0])
        QUANTITIES.append(data[i][1])
        
    print()
    
    ITEM_NAMES = []
    ITEM_PRICES = []
    
    for i in range(len(PRODUCT_IDs)):
        data = c.execute("SELECT Product_Name, Price FROM Product_Info WHERE Product_ID = ?;", (PRODUCT_IDs[i],))
        data = data.fetchone()
        
        ITEM_NAMES.append(data[0])
        ITEM_PRICES.append(data[1])
        
    SUM = 0

    for i in range(len(ITEM_NAMES)):

        SUM = SUM + (QUANTITIES[i] * ITEM_PRICES[i])

        print("Item name: " + str(ITEM_NAMES[i]))
        print("Quantity: " + str(QUANTITIES[i]))
        print("Unit Price: " + str(ITEM_PRICES[i]))

    print()
    print("Total Spent: " + str(SUM))

### Scenario 5

(Someone) realised that he/she forgot to pay for (an item/some items) he/she bought (one day). He/she returns to the store to pay for it and now we need to update the database.

### Scenario 6

On (a day), (a person) comes to the store and buys (a number) of (items).

Let's record this in the database

In [ ]:
def new_user(NAME):
    c.execute("INSERT INTO Customer_Detail (Name) VALUES (?)", (NAME,))
    
def new_item(ITEM):
    PRICE = input("New item, please enter price: $")
    c.execute("INSERT INTO Product_Info (Product_Name, Price) VALUES (?,?)", (ITEM, PRICE,))

def new_order(NAME, DAY, QUANTITY, ITEM):
    CUSTOMER_ID = c.execute("SELECT Customer_ID FROM Customer_Detail WHERE Name = ?", (NAME,))
    CUSTOMER_ID = CUSTOMER_ID.fetchone()
    CUSTOMER_ID = CUSTOMER_ID[0]
    
    PRODUCT_ID = c.execute("SELECT Product_ID FROM Product_Info WHERE Product_Name = ?", (ITEM,))
    PRODUCT_ID = PRODUCT_ID.fetchone()
    PRODUCT_ID = PRODUCT_ID[0]
    
    c.execute("INSERT INTO Order_Record (Product_ID, Quantity, Customer_ID, Day) VALUES (?,?,?,?)", (PRODUCT_ID, QUANTITY, CUSTOMER_ID, DAY,))

def new_record(NAME, DAY, QUANTITY, ITEM):
    exist = c.execute("SELECT count(*) FROM Customer_Detail WHERE Name = ?", (NAME,))
    exist = exist.fetchone()
    
    if exist[0] == 0:
        new_user(NAME)
    else:
        pass
    
    exist = c.execute("SELECT count(*) FROM Product_Info WHERE Product_Name = ?", (ITEM,))
    exist = exist.fetchone()
    
    if exist[0] == 0:
        new_item(ITEM)
    else:
        pass
    
    new_order(NAME, DAY, QUANTITY, ITEM)
    
    conn.commit()